# 📊 Context Recall
The key is having context that supports all parts of the answer.
What It Measures: Context recall is all about making sure the retrieved context is in sync with the ground truth answer. You're aiming for a perfect match here.

Scoring: It's scored on a scale of 0 to 1. The closer you get to 1, the better your model is at aligning the context with the answer.

How It Works: The model breaks down the ground truth answer sentence by sentence. For each sentence, it checks if it can be traced back to the retrieved context. Ideally, every sentence should have a corresponding piece of context.

Example:

❓ Question: Where is France and what is its capital?

🟢 Ground Truth: France is in Western Europe and its capital is Paris.

⬆️ High Recall: France, in Western Europe. Paris, its capital.

⬇️ Low Recall: France, in Western Europe, known for wines and cuisine.